In [2]:
import os
import io
import logging
import time
import requests
from dotenv import load_dotenv
from pathlib import Path
from docling.document_converter import DocumentConverter, DocumentStream, PdfFormatOption
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions

load_dotenv()

True

In [3]:
# Autenticação Microsoft Graph
TENANT_ID = os.getenv('TENANT_ID')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

token_url = f'https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token'

data = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'scope': 'https://graph.microsoft.com/.default'
}

response = requests.post(token_url, data=data)

access_token = response.json()['access_token']

print(access_token)

eyJ0eXAiOiJKV1QiLCJub25jZSI6IlBsZ1dDMEpsV2FieC16Qjc3NnRyY0UzYWtWaWtVd0VLRmdTYklqSFpvdU0iLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC84YTYwZTJlMC1jMGRlLTQxODItYWZlMy03NzE3NjdlYTYwMmUvIiwiaWF0IjoxNzU2MDU1OTQ4LCJuYmYiOjE3NTYwNTU5NDgsImV4cCI6MTc1NjA1OTg0OCwiYWlvIjoiazJSZ1lBZ09zaXBwTDM3aEZ2cGNiMWxCODRZNkFBPT0iLCJhcHBfZGlzcGxheW5hbWUiOiJuOG4tc2hhcmVwb2ludC1pbnRlZ3JhdGlvbiIsImFwcGlkIjoiNTQ5OTIyNmQtYjAxZS00NDE5LThhZTYtMzAxMTIwMzVmNTIwIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvOGE2MGUyZTAtYzBkZS00MTgyLWFmZTMtNzcxNzY3ZWE2MDJlLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMDc2YmYyZDItOWNhMC00NGRiLWE1YzUtZDhkMWQ3YzNmZjRiIiwicmgiOiIxLkFXNEE0T0pnaXQ3QWdrR3Y0M2NYWi1wZ0xnTUFBQUFBQUFBQXdBQUFBQUFBQUFCdUFBQnVBQS4iLCJyb2xlcyI6WyJTaXRlcy5SZWFkLkFsbCIsIlNpdGVzLlJlYWRXcml0ZS5BbGwiLCJGaWxlcy5SZWFkV3JpdGUuQWxsIiwiRmlsZXMuUmVhZC5BbGwiLCJTaXRlcy5

In [103]:
SITE_SHAREPOINT = os.getenv('SITE_SHAREPOINT')

response_site = requests.get(
    f'https://graph.microsoft.com/v1.0/sites/taticogestao.sharepoint.com:/sites/{SITE_SHAREPOINT}',
    headers={
        'Authorization': f'Bearer {access_token}'
    }
)

site_id = response_site.json().get("id")

print(site_id)

taticogestao.sharepoint.com,d22b24f0-c48c-4852-bc8f-08115334c235,cd8014fe-7126-4ac2-9e58-a7accf46ee98


In [104]:
response_drive = requests.get(
    f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive",
    headers={"Authorization": f"Bearer {access_token}"}
)

drive_id = response_drive.json()['id']

print(drive_id)

b!8CQr0ozEUki8jwgRUzTCNf4UgM0mccJKnlinrM9G7pjRh2kC-8VBTo74zw6OsD1t


In [105]:
FOLDER_SHAREPOINT = os.getenv('FOLDER_SHAREPOINT')

response_files = requests.get(
    f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{FOLDER_SHAREPOINT}:/children",
    headers={"Authorization": f"Bearer {access_token}"}  
)

In [106]:
items = response_files.json().get("value", [])
folders = [item for item in items if "folder" in item]

pdf_files = []

# Para cada pasta (ano), lista os arquivos
for folder in folders:
    folder_name = folder["name"]
    folder_path = f"{FOLDER_SHAREPOINT}/{folder_name}"
    
    response_sub = requests.get(
        f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{folder_path}:/children",
        headers={"Authorization": f"Bearer {access_token}"}
    )

    files_in_folder = response_sub.json().get("value", [])

    for file in files_in_folder:
        file_name = file["name"]
        download_url = file.get("@microsoft.graph.downloadUrl")
        if not download_url:
            continue

        file_resp = requests.get(download_url)
        if file_resp.status_code == 200:
            pdf_files.append({
                "folder": folder_name,      # em qual ano estava
                "file_name": file_name,     # nome do arquivo
                "content": file_resp.content  # conteúdo em bytes
            })
        else:
            print(f"Erro ao baixar {file_name}: {file_resp.status_code}")

In [81]:
pdf_bytes = pdf_files[64]['content']

pdf_bytes

b'%PDF-1.7\n\n4 0 obj\n<<\n/BitsPerComponent 8\n/ColorSpace /DeviceRGB\n/Filter /DCTDecode\n/Height 101\n/Length 1334\n/Subtype /Image\n/Type /XObject\n/Width 101\n>>\nstream\n\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\r\t\n\x0b\n\x08\r\x0b\x0b\x0b\x0f\x0e\r\x10\x14!\x15\x14\x12\x12\x14(\x1d\x1e\x18!0*21/*.-4;K@48G9-.BYBGNPTUT3?]c\\RbKSTQ\xff\xdb\x00C\x01\x0e\x0f\x0f\x14\x11\x14\'\x15\x15\'Q6.6QQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ\xff\xc0\x00\x11\x08\x00e\x00e\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9

In [107]:
pdf = [pdf_files[60]]

In [83]:
converter = DocumentConverter()

stream = io.BytesIO(pdf_bytes)

doc_stream = DocumentStream(stream=stream, file_type='pdf', name='temp.pdf')

result = converter.convert(doc_stream)

markdown = result.document.export_to_markdown()

INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 5a43216f093a7c32c3d8090bbb471faa
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.pipeline.base_pipeline:Processing document temp.pdf
INFO:docling.document_converter:Finished converting document temp.pdf in 8.02 sec.


In [84]:
with open('teste.md', 'w', encoding='utf-8') as f:
    f.write(markdown)

In [ ]:
IMAGE_RESOLUTION_SCALE = 2.0
_log = logging.getLogger(__name__)


def process_pdf_in_memory(pdf_bytes: bytes, file_name: str, output_dir: Path):
    """
    Converte e exporta imagens/markdown/HTML de um PDF em memória (bytes).
    """
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    # ✅ Cria DocumentStream a partir dos bytes
    doc_stream = DocumentStream(
        name=file_name,
        media_type="application/pdf",
        stream=io.BytesIO(pdf_bytes)
    )

    # Agora sim converte
    conv_res = doc_converter.convert(doc_stream)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = Path(file_name).stem

    # Salva imagens das páginas
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

    # Salva imagens de tabelas e figuras
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

    # Exporta markdown e HTML
    conv_res.document.save_as_markdown(
        output_dir / f"{doc_filename}-with-images.md",
        image_mode=ImageRefMode.EMBEDDED
    )
    conv_res.document.save_as_markdown(
        output_dir / f"{doc_filename}-with-image-refs.md",
        image_mode=ImageRefMode.REFERENCED
    )
    conv_res.document.save_as_html(
        output_dir / f"{doc_filename}-with-image-refs.html",
        image_mode=ImageRefMode.REFERENCED
    )

    end_time = time.time() - start_time
    _log.info(f"{file_name} convertido e exportado em {end_time:.2f} segundos.")


def main(pdf_files: list[dict]):
    output_dir = Path("images")
    for pdf in pdf_files:
        process_pdf_in_memory(pdf["content"], pdf["file_name"], output_dir)


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    main(pdf)

INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash a9924d9e864af6d91e14eae28d2791b8
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.pipeline.base_pipeline:Processing document 2025.03 - RelatorioMensal.pdf
INFO:docling.document_converter:Finished converting document 2025.03 - RelatorioMensal.pdf in 9.41 sec.
INFO:__main__:2025.03 - RelatorioMensal.pdf convertido e exportado em 9.63 segundos.


In [ ]:
IMAGE_RESOLUTION_SCALE = 2.0
_log = logging.getLogger(__name__)


def process_pdf_in_memory(pdf_bytes: bytes, file_name: str, output_dir: Path):
    """
    Converte e exporta imagens/markdown/HTML de um PDF em memória (bytes).
    """
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    # ✅ Cria DocumentStream a partir dos bytes
    doc_stream = DocumentStream(
        name=file_name,
        media_type="application/pdf",
        stream=io.BytesIO(pdf_bytes)
    )

    # Agora sim converte
    conv_res = doc_converter.convert(doc_stream)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = Path(file_name).stem

    # Salva imagens das páginas
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

def main(pdf_files: list[dict]):
    output_dir = Path("images")
    for pdf in pdf_files:
        process_pdf_in_memory(pdf["content"], pdf["file_name"], output_dir)


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    main(pdf)

INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash a9924d9e864af6d91e14eae28d2791b8
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.pipeline.base_pipeline:Processing document 2025.03 - RelatorioMensal.pdf
INFO:docling.document_converter:Finished converting document 2025.03 - RelatorioMensal.pdf in 9.95 sec.


In [5]:
PATH = 'images/2025.03 - RelatorioMensal-1.png'

converter = DocumentConverter()

result = converter.convert(PATH)

markdown = result.document.export_to_markdown()

with open('teste.md', 'w', encoding='utf-8') as f:
    f.write(markdown)

c:\Users\viva_\workspace\cezarmaldini\agent-reports-crm\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
